## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [3]:
convention_db = sqlite3.connect("/Users/kyledalope/Downloads/nb-assignment-data/2020_Conventions.db") #connects to db
convention_cur = convention_db.cursor() #cursor object from db, to execute queries

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [8]:
#Identify table names 

convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = convention_cur.fetchall()

# Check if there are any tables
if len(tables) == 0:
    print("No tables found in the database.")
else:
    # Print the table names
    for table in tables:
        print(table[0])  

conventions


In [10]:
#identify column names of conventions table

convention_cur.execute("PRAGMA table_info(conventions)")
columns = convention_cur.fetchall()

# Check if there are any columns
if len(columns) == 0:
    print("No columns found in the table.")
else:
    # Print the column names
    for column in columns:
        print(column[1])


party
night
speaker
speaker_count
time
text
text_len
file


In [11]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            ''')

for row in query_results :
    # store the results in convention_data
    text = row[0]
    party = row[1]
    convention_data.append([text, party])
    pass # remove this
    


Let's look at some random entries and see if they look right. 

In [12]:
random.choices(convention_data,k=10)

[['Thank you Maria and Barney. It’s my pleasure to call on Platform Committee co-chairs Julie Chavez Rodriguez and Denis McDonough to present their committee’s report.',
  'Democratic'],
 ['And beckoned by the lady behind us, we will reform our immigration system so that immigrants yearning to breathe free, will at last become American citizens. Together, we can reignite the hope once felt by millions of men and women, huddled masses on creaking ships, who glimpsed this mighty woman with her torch, knowing they could build a better life here in America. And out of this long, national nightmare, America will finally awaken to a brighter future and a new day.',
  'Democratic'],
 ['He was a lifeguard along with the Black lifeguards. That’s when I first seen Joe and we became friends.',
  'Democratic'],
 ['We have 10 to 12 employees on a given day. When COVID-19 hit it was a lot of confusion.',
  'Democratic'],
 ['(singing.', 'Democratic'],
 ['There is a long way to go. But if this much ca

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [13]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2891 as features in the model.


In [14]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    
    return(ret_dict)

In [ ]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [ ]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [ ]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [ ]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(25)

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [ ]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [ ]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [ ]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [ ]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [ ]:

for tweet, party in tweet_data_sample :
    estimated_party = 'Gotta fill this in'
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [ ]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [ ]:
results

### Reflections

_Write a little about what you see in the results_ 

# References

How do I trim whitespace? (n.d.). Stack Overflow. Retrieved June 3, 2023, from https://stackoverflow.com/questions/1185524/how-do-i-trim-whitespace

‌sqlite3 — DB-API 2.0 interface for SQLite databases — Python 3.8.2 documentation. (n.d.). Docs.python.org. https://docs.python.org/3/library/sqlite3.html

‌